<a href="https://colab.research.google.com/github/grymoon5/Burn-Area-Mapping-and-Fire-Spread-Potential-Modeling-Using-SAR-and-Open-Data/blob/main/SAR_Based_Burn_Area_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install rasterio numpy matplotlib scikit-image geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 56.7 MB/s eta 0:00:00


In [3]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load pre and post images
with rasterio.open('pre_fire.tif') as src:
    pre = src.read(1)
    profile = src.profile

with rasterio.open('post_fire.tif') as src:
    post = src.read(1)


RasterioIOError: pre_fire.tif: No such file or directory

In [ ]:
# Log-ratio burn detection
change = np.log1p(post) - np.log1p(pre)

In [ ]:
# Simple thresholding (can refine later)
burn_mask = change < -1  # you can experiment with threshold

In [ ]:
# Save burn mask
with rasterio.open('burn_mask.tif', 'w', **profile) as dst:
    dst.write(burn_mask.astype(rasterio.uint8), 1)

In [ ]:
# Visualize
plt.figure(figsize=(12,4))
plt.subplot(1,3,1); plt.imshow(pre, cmap='gray'); plt.title('Before')
plt.subplot(1,3,2); plt.imshow(post, cmap='gray'); plt.title('After')
plt.subplot(1,3,3); plt.imshow(burn_mask, cmap='Reds'); plt.title('Burn Area')
plt.tight_layout()
plt.show()

In [ ]:
from skimage.filters import gaussian

# Assume wind_dir, slope, vegetation are normalized layers
spread = (0.5 * wind_dir) + (0.3 * slope) + (0.2 * vegetation)

# Smooth it slightly
spread_smoothed = gaussian(spread, sigma=1)

# Visualize
plt.imshow(spread_smoothed, cmap='hot')
plt.colorbar(label="Spread Potential")
plt.title("Predicted Fire Spread Potential")
plt.show()